In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
# Cài đặt Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes



In [ ]:
try:
    import unsloth
    import datasets
    print("Thư viện đã cài thành công.")
    print(f"Unsloth version: {unsloth.__version__}")
except ImportError:
    print("Chưa cài thư viện")

Thư viện đã cài thành công.
Unsloth version: 2025.11.6


In [ ]:
from unsloth import FastLanguageModel
import torch

# --- CẤU HÌNH  ---
max_seq_length = 2048
dtype = None # None để tự động chọn float16 cho T4 GPU
load_in_4bit = True

# 1. LOAD BASE MODEL
# dùng Qwen-1.5B
print("Đang tải Model Qwen-1.5B-Instruct...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. CẤU HÌNH LORA
print("Đang cấu hình LoRA Adapter...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], Các lớp mạng cần train
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("HOÀN TẤT: Model và LoRA đã sẵn sàng!")

Đang tải Model Qwen-1.5B-Instruct...
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

⚙️ Đang cấu hình LoRA Adapter...


Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


HOÀN TẤT: Model và LoRA đã sẵn sàng!


In [ ]:
from datasets import Dataset
import json
import os

# --- CẤU HÌNH ĐƯỜNG DẪN ---
input_file = '/content/drive/MyDrive/My_Chatbot_Data/raw_history.json'

print(f"Đang tải và xử lý dữ liệu từ {input_file}...")

# 1. ĐỌC FILE & LỌC DỮ LIỆU
data_points = []
error_count = 0

if not os.path.exists(input_file):
    print(f"LỖI: Không tìm thấy file tại {input_file}")
else:
    with open(input_file, 'r', encoding='utf-8') as f:
        # Hỗ trợ đọc từng dòng (JSONL) hoặc đọc cả cục (JSON)
        lines = f.readlines()

        # Nếu file là json array [], nối lại để parse, còn không thì duyệt từng dòng
        try:
            full_data = json.loads("".join(lines))
            if isinstance(full_data, list): items = full_data
            else: items = [full_data]
        except:
            items = [json.loads(line) for line in lines if line.strip()]

    for item in items:
        try:
            messages = item.get('messages', [])

            system = "Bạn là trợ lý am hiểu lịch sử Việt Nam."
            user = ""
            final_answer = ""

            for msg in messages:
                role = msg.get('role')
                content = msg.get('content')
                channel = msg.get('channel', '')

                if role == 'system': system = content
                elif role == 'user': user = content
                elif role == 'assistant':
                    # Logic lọc: Chỉ lấy channel='final' hoặc không có channel
                    if channel == 'final' or channel == '':
                        final_answer = content

            # Chỉ lấy mẫu đủ bộ
            if user and final_answer:
                # Tạo cấu trúc hội thoại
                conversation = [
                    {"role": "system", "content": system},
                    {"role": "user", "content": user},
                    {"role": "assistant", "content": final_answer}
                ]
                data_points.append({"conversations": conversation})
        except:
            error_count += 1
            continue

    print(f"   -> Tìm thấy {len(data_points)} hội thoại hợp lệ.")

    # 2. TẠO DATASET OBJECT
    dataset = Dataset.from_list(data_points)

    # 3. FORMAT DỮ LIỆU
    # Chuyển đổi list hội thoại thành chuỗi văn bản theo format của Qwen
    def formatting_prompts_func(examples):
        conversations = examples["conversations"]
        texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=False) for conv in conversations]
        return { "text": texts }

    # Áp dụng format
    dataset = dataset.map(formatting_prompts_func, batched=True)

    print(f"HOÀN TẤT! Dữ liệu đã sẵn sàng để train.")
    print("-" * 30)
    print("Ví dụ mẫu đầu tiên sau khi xử lý:")
    print(dataset[0]['text'][:500] + "...") # In thử 500 ký tự đầu

Đang tải và xử lý dữ liệu từ /content/drive/MyDrive/My_Chatbot_Data/raw_history.json...
   -> Tìm thấy 15000 hội thoại hợp lệ.


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

HOÀN TẤT! Dữ liệu đã sẵn sàng để train.
------------------------------
Ví dụ mẫu đầu tiên sau khi xử lý:
<|im_start|>system
Bạn là trợ lý am hiểu lịch sử Việt Nam. Trả lời bằng tiếng Việt, chính xác, súc tích; nêu mốc thời gian và nhân vật then chốt khi phù hợp.<|im_end|>
<|im_start|>user
Vì sao xảy ra sự kiện Đặt ách bảo hộ toàn cõi (1884)? Nêu bối cảnh và nguyên nhân chính.<|im_end|>
<|im_start|>assistant
Bối cảnh trước 1884: Việt Nam vào Liên bang Đông Dương là phản ứng trước sức ép thời cuộc. Nguyên nhân chính gồm mâu thuẫn quyền lực, xâm lược/đe dọa bên ngoài và nhu cầu khẳng định chủ quyền. K...


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# --- CẤU HÌNH ĐƯỜNG DẪN LƯU MODEL ---
output_dir = "/content/drive/MyDrive/Saved_Model_Finetune"

# --- THIẾT LẬP THAM SỐ HUẤN LUYỆN ---
training_args = TrainingArguments(
    per_device_train_batch_size = 2,  # Batch size an toàn cho T4
    gradient_accumulation_steps = 4,  # Tương đương batch size 8
    warmup_steps = 5,

    # Cấu hình Epoch & Lưu trữ
    num_train_epochs = 3,             # Train 3 epochs
    save_strategy = "epoch",          # Lưu checkpoint sau mỗi Epoch

    learning_rate = 2e-4,             # Tốc độ học
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 50,               # In loss mỗi 50 bước
    optim = "adamw_8bit",             # Tối ưu hóa tiết kiệm bộ nhớ
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,

    output_dir = output_dir,
    save_total_limit = 2,             # Chỉ giữ 2 bản mới nhất
    report_to = "none",
)

# --- KHỞI TẠO TRAINER ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = training_args,
)

# --- BẮT ĐẦU TRAIN ---
print("ĐANG BẮT ĐẦU HUẤN LUYỆN...")
print(f"   - Số lượng dữ liệu: {len(dataset)} mẫu")
print(f"   - Nơi lưu model: {output_dir}")
print("Không tắt Tab trình duyệt.")

trainer_stats = trainer.train()

print("HUẤN LUYỆN HOÀN TẤT! Model đã được lưu vào Saved_Model_Finetune.")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/15000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


ĐANG BẮT ĐẦU HUẤN LUYỆN...
   - Số lượng dữ liệu: 15000 mẫu
   - Nơi lưu model: /content/drive/MyDrive/Saved_Model_Finetune
Không tắt Tab trình duyệt.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15,000 | Num Epochs = 3 | Total steps = 5,625
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss
50,1.170600
100,0.235600
150,0.104600
200,0.073900
250,0.062800
300,0.058400
350,0.056700
400,0.054400
450,0.053400
500,0.055500


HUẤN LUYỆN HOÀN TẤT! Model đã được lưu vào Saved_Model_Finetune.
